### COSC102 Assignment 3 - Prepared Data
The purpose of this notebook is to prepare the data for the model training.
We will use the data from the IMU sensor to train a model to predict the activity of the user.



In [ ]:
import csv
import matplotlib.pyplot as plt
import math
import numpy as np
import statistics
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import plot_confusion_matrix

# Set some lists to store the IMU data.

time_track = []
ax_set = []
ay_set = []
az_set = []

gx_set = []
gy_set = []
gz_set = []

activity_set = []

def map_activity(activity_string):
    if activity_string == 'Standing':
        return 0
    elif activity_string == 'Walking':
        return 1
    elif activity_string == 'Jogging':
        return 2
    elif activity_string == 'Side-Step':
        return 3
    elif activity_string == 'Running':
        return 4


In [ ]:
# Read the IMU data from the csv file.
with open ('./datasets/a3_imu_data.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        time_track.append(float(row[0]));
        ax_set.append(float(row[1]))
        ay_set.append(float(row[2]))
        az_set.append(float(row[3]))
        gx_set.append(float(row[4]))
        gy_set.append(float(row[5]))
        gz_set.append(float(row[6]))

In [ ]:
# Check the raw IMU data from the csv file.

fig, axs = plt.subplots(2,1,figsize=(20,10))
axs[0].plot(ax_set, color='r')
axs[0].plot(ay_set, color='g')
axs[0].plot(az_set, color='b')

axs[1].plot(gx_set, color='r')
axs[1].plot(gy_set, color='g')
axs[1].plot(gz_set, color='b')

In [ ]:
start_idx = 800

start_ts = time_track[start_idx]
print('start_ts: ', start_ts)

end_idx = time_track.index(start_ts + 711)

# Sanity Check
fig, axs = plt.subplots(2,1,figsize=(20,10))
axs[0].plot(ax_set[start_idx:end_idx], color='r')
axs[0].plot(ay_set[start_idx:end_idx], color='g')
axs[0].plot(az_set[start_idx:end_idx], color='b')

axs[1].plot(gx_set[start_idx:end_idx], color='r')
axs[1].plot(gy_set[start_idx:end_idx], color='g')
axs[1].plot(gz_set[start_idx:end_idx], color='b')

In [ ]:

# Add activity annotation to the IMU data from the csv file.
with open ('./datasets/a3_activity_annotations.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None) # skip the headers
    for row in reader:
        time_stamp = (row[0].split('='))[1]
        activity_set.append([float(time_stamp),row[-1], map_activity(row[-1])])

In [ ]:
# Now we can create an activity time track for each data point within
# the imu timeseries. We will have a list to store the numeric code and the string.
activity_timeseries = [];
activity_string_timeseries = [];
activity_idx = 0;

# We need the time stamp for the start point
start_time = time_track[start_idx]

# The time track segment we are interested in 
time_track_segment = time_track[start_idx:end_idx]

# #Add an 'end' activity - this book-ends the data
activity_set.append([time_track_segment[-1]-start_time, 'Standing'])

# Zero out the time track segment to make it match the video time
time_track_segment = np.array(time_track_segment)-start_time

ig, ax = plt.subplots(figsize=(20,10))
ax.plot(time_track_segment,ax_set[start_idx:end_idx], color='r')
ax.plot(time_track_segment,ay_set[start_idx:end_idx], color='g')
ax.plot(time_track_segment,az_set[start_idx:end_idx], color='b')


for imu_time_track_item in time_track_segment:
    current_time =  imu_time_track_item
    next_activity_ts = activity_set[activity_idx+1][0]
    
    # Here we need to move to the next activity in the annotations data if the current
    # IMU data point lies after the next annotation time stamp.
    if current_time > next_activity_ts:
        #Move to nex activity
        activity_idx = activity_idx + 1;
        next_activity_ts = activity_set[activity_idx+1][0]
    
    
    activity_timeseries.append(activity_set[activity_idx][2])
    activity_string_timeseries.append(activity_set[activity_idx][1])

# Plot the location of the activity transitions just to sanity check the data
for act in np.array(list(zip(*activity_set)))[0,:]:
    ax.axvline(float(act))

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.hist(activity_timeseries, bins=[-0.5,0.5,1.5,2.5,3.5, 4.5] , rwidth=0.5, align='mid')

In [ ]:
#We are only interested in the video window
ax_set = ax_set[start_idx:end_idx]
ay_set = ay_set[start_idx:end_idx]
az_set = az_set[start_idx:end_idx]
gx_set = gx_set[start_idx:end_idx]
gy_set = gy_set[start_idx:end_idx]
gz_set = gz_set[start_idx:end_idx]

In [ ]:
# Now we can start working on the machine learning workflow.
# The first step is to calculate some feature that we can use.
# We will base our analysis around a moving window across the
# the timeseries data. This involves taking a fixed length window, based upon the time, and
# sliding it across the data.
# To keep thing simple, we will start with a 1 second window with no overlapping
# data points

feature_set = []
target_set  = []            
window_size = 1.0;
bad_data_point = [104, 105, 163, 164, 274, 275]            

for t in range(int(time_track_segment[0]),int(time_track_segment[-1])):
    
    #There is a missing data-point (probably due to a bad segment on the SD card)
    
    if t in bad_data_point:
      continue
    
    #The index function finds the index of the first occurnace of the data
    window_start_idx = list(time_track_segment).index(t)
    window_end_idx = list(time_track_segment).index(t+window_size)
    ax_window = ax_set[window_start_idx:window_end_idx]
    ay_window = ay_set[window_start_idx:window_end_idx]
    az_window = az_set[window_start_idx:window_end_idx]
    gx_window = gx_set[window_start_idx:window_end_idx]
    gy_window = gy_set[window_start_idx:window_end_idx]
    gz_window = gz_set[window_start_idx:window_end_idx]

    #activity that will be assigned to the set of features
    activity_code = activity_timeseries[window_start_idx]

    # Now we can build features from the data window
    # Mean
    mu_ax = statistics.mean(ax_window)
    mu_ay = statistics.mean(ay_window)
    mu_az = statistics.mean(az_window)
    mu_gx = statistics.mean(gx_window)
    mu_gy = statistics.mean(gy_window)
    mu_gz = statistics.mean(gz_window)
    
    # Max
    max_ax = max(ax_window)
    max_ay = max(ay_window)
    max_az = max(az_window)
    max_gx = max(gx_window)
    max_gy = max(gy_window)
    max_gz = max(gz_window)
    
    # Min
    min_ax = min(ax_window)
    min_ay = min(ay_window)
    min_az = min(az_window)
    min_gx = min(gx_window)
    min_gy = min(gy_window)
    min_gz = min(gz_window)
    
    ax_abs_sum = 0
    ay_abs_sum = 0
    az_abs_sum = 0
    
    gx_abs_sum = 0
    gy_abs_sum = 0
    gz_abs_sum = 0
    
    a_sum_sq = 0
    g_sum_sq = 0
    
    # Here we need to accumulate the values to calculate the SMA and AI
    for i in range(0, len(ax_window)):
        
        # Add up the absolute values for the SMA
        ax_abs_sum = ax_abs_sum + abs(ax_window[i])
        ay_abs_sum = ay_abs_sum + abs(ay_window[i])
        az_abs_sum = az_abs_sum + abs(az_window[i])
        
        gx_abs_sum = gx_abs_sum + abs(gx_window[i])
        gy_abs_sum = gy_abs_sum + abs(gy_window[i])
        gz_abs_sum = gz_abs_sum + abs(gz_window[i])
        
        
        a_sum_sq = ((ax_window[i]**2) + (ay_window[i]**2) + (az_window[i]**2)) + a_sum_sq
        g_sum_sq = ((gx_window[i]**2) + (gy_window[i]**2) + (gz_window[i]**2)) + g_sum_sq
    
    # Signal Magnitude area    
    a_sma = (ax_abs_sum + ay_abs_sum + az_abs_sum) / len(ax_window)
    g_sma = (gx_abs_sum + gy_abs_sum + gz_abs_sum) / len(ax_window)

    # Average intensity
    a_av_intensity = math.sqrt(a_sum_sq) / len(ax_window)
    g_av_intensity = math.sqrt(g_sum_sq) / len(ax_window)


    feature_row = [mu_ax, mu_ay, mu_az, mu_gx, mu_gy, mu_gz, \
                   max_ax, max_ay, max_az, max_gx, max_gy, max_gz, \
                   min_ax, min_ay, min_az, min_gx, min_gy, min_gz, \
                   a_sma, g_sma, a_av_intensity, g_av_intensity]

    feature_set.append(feature_row)
    target_set.append(activity_code)
    feature_names = ['mu_ax', 'mu_ay', 'mu_az', 'mu_gx', 'mu_gy', 'mu_gz', \
                    'max_ax', 'max_ay', 'max_az', 'max_gx', 'max_gy', 'max_gz', \
                    'min_ax', 'min_ay', 'min_az', 'min_gx', 'min_gy', 'min_gz', \
                    'a_sma', 'g_sma', 'a_av_intensity', 'g_av_intensity']
                    


In [ ]:
# Lets visualise some of the features just out of interest - we will look at the SMA and AI
plot_range = range(1,705)
fig, ax = plt.subplots(figsize=(20,10))
ax.plot(plot_range,np.array(feature_set)[:,18], color='r')
ax.plot(plot_range,np.array(feature_set)[:,19], color='g')
ax.plot(plot_range,np.array(feature_set)[:,20], color='b')
ax.plot(plot_range,np.array(feature_set)[:,21], color='c')

# Plot the location of the activity transitions just to sanity check the data
for act in np.array(list(zip(*activity_set)))[0,:]:
    ax.axvline(float(act), alpha=0.5)
    

In [ ]:
### COSC102 Assignment 3 - KNN Classifier


In [ ]:
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import seaborn as sns

feature_set = np.array(feature_set)
target_set  = np.array(target_set)

#Normalisation of data to minimize outliers
features_set = preprocessing.MinMaxScaler().fit_transform(feature_set)
# Train test split to be fitted            
X_train, X_test, y_train, y_test = train_test_split(feature_set, target_set, test_size=0.2, random_state=42)


In [ ]:
## An attempt at optomization for K - kept resulting in 0 (k cannot equal 0) 


# k_values = [i for i in range (1,31)]
# scores = []

# for k in k_values:
#     knn = neighbors.KNeighborsClassifier(n_neighbors=k)
#     score = cross_val_score(knn, feature_set, target_set, cv=5)
#     scores.append(np.mean(score))
    
# best_index = np.argmax(scores)
# best_k = k_values[best_index]

# sns.lineplot(x = k_values, y = scores, marker = 'o')
# plt.xlabel("K Values")
# plt.ylabel("Accuracy Score") 

In [ ]:
 # Optimisation of k using accuracy score
    # Also includes mean squared error

best_k = 0
best_mean_f1 = 0
f1_set = np.zeros(20)
best_k_range = range(3,20) # remove k = 1 and 2 as they are outliers and cause overfitting

for k in best_k_range:
   #uses p = 1 for distance
  clf = neighbors.KNeighborsClassifier(k, weights='uniform', p=1).fit(X_train, y_train)
  # Return the predictions for the 5-Fold crossvalidation
  y_predicted = cross_val_predict(clf, X_train,y_train, cv=5)

  # Print out the recall, precision and F1 scores
  print("K: ", k)
  print("F1 Score:",f1_score(y_train,y_predicted,average=None))
  print("Accuracy:",str(accuracy_score(y_train, y_predicted)*100)+"%")
  print("Mean squared error: ",str(np.mean((y_train - y_predicted) ** 2)*100)+"%") 

  # Test
  f1_set[k] = np.mean(f1_score(y_train,y_predicted,average=None))
  current_f1 = np.mean(f1_score(y_train,y_predicted,average=None))
  
  if current_f1 > best_mean_f1:
    best_mean_f1 = current_f1
    best_k = k
    best_clf = clf

In [ ]:
plt.plot(best_k_range, f1_set[3:20])
plt.xlabel('K')
plt.xticks(np.arange(3, 20, 1))
plt.ylabel('F1 Score')
plt.title('F1 Score vs K')
plt.show()

In [ ]:
##Plot confusion matrices

best_clf = neighbors.KNeighborsClassifier(best_k, weights='uniform', p=1).fit(X_train, y_train)

# Construct the confusion matricies
conf_mat_train = confusion_matrix(y_train, y_predicted)

# Plot the confusion matrices
fig, ax = plt.subplots()
disp = plot_confusion_matrix(best_clf, X_train, y_train,
                                 display_labels=['Standing', 'Walking','Jogging', 'Side-Step','Running'],
                                 cmap=plt.cm.Blues,ax=ax)
# Rotate the labels so they can be read
for tick in ax.get_xticklabels():
    tick.set_rotation(45)

ax.set_title('5-Fold Training - Best K')

fig, ax = plt.subplots()
disp = plot_confusion_matrix(best_clf, X_test, y_test,
                                 display_labels=['Standing', 'Walking','Jogging', 'Side-Step','Running'],
                                 cmap=plt.cm.Blues,ax=ax)
# Rotate the labels so they can be read
for tick in ax.get_xticklabels():
    tick.set_rotation(45)

ax.set_title('Testing Set')

In [ ]:
# Calculate the ROC and AUC for each class
y_score = best_clf.predict_proba(X_test)
y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3, 4])
n_classes = y_test_bin.shape[1]
class_names = ['Standing', 'Walking','Jogging', 'Side-Step','Running']

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curves
fig, ax = plt.subplots()
for i in range(n_classes):
    ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for class %s' % (roc_auc[i], class_names[i]))
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.legend(loc="lower right")
ax.set_title('Receiver operating characteristic for KNN')




In [ ]:
#Plot 2 x Visuals of KNN with test data
#Uses 

import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, neighbors
from matplotlib.colors import ListedColormap

#list of features names
f_names = ["Mean Ax", "Mean Ay", "Mean Az","Mean Gx", "Mean Gy", "Mean Gz",
           "Max Ax", "Max Ay", "Max Az","Max Gx", "Max Gy", "Max Gz", 
           "Min Ax", "Min Ay", "Min Az","Min Gx", "Min Gy", "Min Gz",
           "Acceleration SMA", "Gyroscope SMA" , "Acceleration AI", "Gyroscope AI"]


#plot Acceleration AI vs Gyroscope AI
X = X_test[:,20:22]
y = y_test

viz_clf = neighbors.KNeighborsClassifier(best_k, weights='uniform', p=1).fit(X, y)

h = 50
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue', 'red', 'lightseagreen'])
cmap_bold = ListedColormap(['darkorange', 'c', 'darkblue', 'darkred', 'g'])
 
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
 
Z = viz_clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure()
plt.pcolormesh(xx, yy, Z)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, edgecolor='k', s=20)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("kNN classification (k = %i)" % best_k)
plt.xlabel(f_names[20])
plt.ylabel(f_names[21])
plt.show()

#plot Acceleration SMA vs Gyroscpoce SMA
X = X_test[:,18:20]
y = y_test

viz_clf = neighbors.KNeighborsClassifier(best_k, weights='uniform', p=1).fit(X, y)

h = 50
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue', 'red', 'lightseagreen'])
cmap_bold = ListedColormap(['darkorange', 'c', 'darkblue', 'darkred', 'g'])
 
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
 
Z = viz_clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure()
plt.pcolormesh(xx, yy, Z)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, edgecolor='k', s=20)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("kNN classification (k = %i)" % best_k)
plt.xlabel(f_names[18])
plt.ylabel(f_names[19])
plt.show()

#For video: shows that those features are not the best but there is decisionn boundaries. 
#When using a higher k (10) it is not as precise